<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Stock_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-3*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')

mypf = mypf[mypf['InPortfolio'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


array(['ASIANPAINT', 'BAJFINANCE', 'BATAINDIA', 'DMART', 'EICHERMOT',
       'GILLETTE', 'HDFCBANK', 'INDIGOPNTS', 'INFY', 'IRCTC', 'JUBLFOOD',
       'PAGEIND', 'PIDILITIND', 'RELIANCE', 'TATACONSUM'], dtype=object)

In [5]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    
    stock_df['Close'] = round(stock_df['Close'],0)     
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    #current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]   
    #tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [6]:
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [7]:
# portfolio stocks

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df['Investment'] = tmp_df['Avg_Cost'] * tmp_df['Shares']
tmp_df['Current'] = tmp_df['Close'] * tmp_df['Shares']
tmp_df['Estimated'] = tmp_df['Max'] * tmp_df['Shares']
tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Estimated P/L%'] = round((tmp_df['Estimated'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df.sort_values(by = 'Current P/L%', ascending=False)

qualified stocks: 15


,Close,20_SMA,50_SMA,200_SMA,Stock,Max,Avg_Cost,Shares,Investment,Current,Estimated,Current P/L%,Estimated P/L%
7,1275.0,1133.0,1072.0,1315.0,INDIGOPNTS.NS,3112.0,1152.30,44.0,50701.20,56100.0,136928.0,10.65,170.07
1,6658.0,6072.0,5971.0,6582.0,BAJFINANCE.NS,7901.0,6044.41,8.0,48355.28,53264.0,63208.0,10.15,30.72
11,41372.0,39321.0,38302.0,44347.0,PAGEIND.NS,53631.0,38201.00,1.0,38201.00,41372.0,53631.0,8.30,40.39
14,789.0,738.0,720.0,765.0,TATACONSUM.NS,875.0,732.53,66.0,48346.98,52074.0,57750.0,7.71,19.45
0,3019.0,2876.0,2837.0,3070.0,ASIANPAINT.NS,3566.0,2856.73,17.0,48564.41,51323.0,60622.0,5.68,24.83
5,4596.0,4412.0,4477.0,4932.0,GILLETTE.NS,5944.0,4353.00,7.0,30471.00,32172.0,41608.0,5.58,36.55
3,3678.0,3508.0,3454.0,3921.0,DMART.NS,5324.0,3501.37,15.0,52520.55,55170.0,79860.0,5.04,52.05
10,466.0,445.0,443.0,528.0,JUBLFOOD.NS,903.0,448.52,112.0,50234.24,52192.0,101136.0,3.90,101.33
13,2472.0,2379.0,2339.0,2472.0,RELIANCE.NS,2811.0,2382.54,22.0,52415.88,54384.0,61842.0,3.75,17.98
4,3393.0,3228.0,3128.0,3302.0,EICHERMOT.NS,3850.0,3275.70,15.0,49135.50,50895.0,57750.0,3.58,17.53


In [8]:
tmp_df.sort_values(by = 'Estimated P/L%', ascending=False)

,Close,20_SMA,50_SMA,200_SMA,Stock,Max,Avg_Cost,Shares,Investment,Current,Estimated,Current P/L%,Estimated P/L%
7,1275.0,1133.0,1072.0,1315.0,INDIGOPNTS.NS,3112.0,1152.30,44.0,50701.20,56100.0,136928.0,10.65,170.07
10,466.0,445.0,443.0,528.0,JUBLFOOD.NS,903.0,448.52,112.0,50234.24,52192.0,101136.0,3.90,101.33
9,628.0,605.0,604.0,661.0,IRCTC.NS,1164.0,618.80,80.0,49504.00,50240.0,93120.0,1.49,88.11
3,3678.0,3508.0,3454.0,3921.0,DMART.NS,5324.0,3501.37,15.0,52520.55,55170.0,79860.0,5.04,52.05
8,1266.0,1290.0,1390.0,1484.0,INFY.NS,1898.0,1252.12,40.0,50084.80,50640.0,75920.0,1.11,51.58
2,1502.0,1452.0,1429.0,1664.0,BATAINDIA.NS,2173.0,1495.89,34.0,50860.26,51068.0,73882.0,0.41,45.26
11,41372.0,39321.0,38302.0,44347.0,PAGEIND.NS,53631.0,38201.00,1.0,38201.00,41372.0,53631.0,8.30,40.39
5,4596.0,4412.0,4477.0,4932.0,GILLETTE.NS,5944.0,4353.00,7.0,30471.00,32172.0,41608.0,5.58,36.55
1,6658.0,6072.0,5971.0,6582.0,BAJFINANCE.NS,7901.0,6044.41,8.0,48355.28,53264.0,63208.0,10.15,30.72
0,3019.0,2876.0,2837.0,3070.0,ASIANPAINT.NS,3566.0,2856.73,17.0,48564.41,51323.0,60622.0,5.68,24.83


In [9]:
investment = round(sum(tmp_df['Avg_Cost']*tmp_df['Shares']),0)
current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
curr_pnl_amount = current-investment
curr_pnl_percentage = round((current-investment)*100/investment,2)

estimate = round(sum(tmp_df['Max']*tmp_df['Shares']),0)
est_pnl_amount = estimate-investment
est_pnl_percentage = round((estimate-investment)*100/investment,2)

print('Investment: ', investment)
print('Current: ', current)
print('Current Profit/Loss: ', curr_pnl_amount)
print('Current Profit/Loss %: ', curr_pnl_percentage)
print('Estimate: ', estimate)
print('Estimate Profit/Loss: ', est_pnl_amount)
print('Estimate Profit/Loss %: ', est_pnl_percentage)

Investment:  718422.0
Current:  749234.0
Current Profit/Loss:  30812.0
Current Profit/Loss %:  4.29
Estimate:  1067137.0
Estimate Profit/Loss:  348715.0
Estimate Profit/Loss %:  48.54


In [10]:
tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)
tmp_df.sort_values(by = 'Weightage%', ascending=False)

,Close,20_SMA,50_SMA,200_SMA,Stock,Max,Avg_Cost,Shares,Investment,Current,Estimated,Current P/L%,Estimated P/L%,Weightage%
3,3678.0,3508.0,3454.0,3921.0,DMART.NS,5324.0,3501.37,15.0,52520.55,55170.0,79860.0,5.04,52.05,7.31
13,2472.0,2379.0,2339.0,2472.0,RELIANCE.NS,2811.0,2382.54,22.0,52415.88,54384.0,61842.0,3.75,17.98,7.30
2,1502.0,1452.0,1429.0,1664.0,BATAINDIA.NS,2173.0,1495.89,34.0,50860.26,51068.0,73882.0,0.41,45.26,7.08
7,1275.0,1133.0,1072.0,1315.0,INDIGOPNTS.NS,3112.0,1152.30,44.0,50701.20,56100.0,136928.0,10.65,170.07,7.06
6,1644.0,1674.0,1626.0,1557.0,HDFCBANK.NS,1728.0,1672.84,30.0,50185.20,49320.0,51840.0,-1.72,3.30,6.99
10,466.0,445.0,443.0,528.0,JUBLFOOD.NS,903.0,448.52,112.0,50234.24,52192.0,101136.0,3.90,101.33,6.99
8,1266.0,1290.0,1390.0,1484.0,INFY.NS,1898.0,1252.12,40.0,50084.80,50640.0,75920.0,1.11,51.58,6.97
9,628.0,605.0,604.0,661.0,IRCTC.NS,1164.0,618.80,80.0,49504.00,50240.0,93120.0,1.49,88.11,6.89
4,3393.0,3228.0,3128.0,3302.0,EICHERMOT.NS,3850.0,3275.70,15.0,49135.50,50895.0,57750.0,3.58,17.53,6.84
12,2451.0,2406.0,2358.0,2533.0,PIDILITIND.NS,2902.0,2442.09,20.0,48841.80,49020.0,58040.0,0.36,18.83,6.80


In [11]:
tmp_df.to_csv('/content/drive/My Drive/data/stocks/Stock-Portfolio-Analysis.csv', index=False)